In [3]:
import requests
import json
import numpy as np
import pandas as pd
import http.server
import socketserver
import threading
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build

PORT = 8000
authorization_code = None

class Handler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        global authorization_code
        if "/callback" in self.path:
            query = self.path.split('?', 1)[1]
            params = {k: v for k, v in (x.split('=') for x in query.split('&') if '=' in x)}
            code = params.get('code')
            self.send_response(200)
            self.send_header("Content-type", "text/html")
            self.end_headers()
            if code:
                self.wfile.write(b"Authorization code received! You can close this window.")
                authorization_code = code
                print(f"Authorization code: {code}")
            else:
                self.wfile.write(b"Authorization code not found in the callback URL.")

# Define necessary credentials and URLs
client_id = '1000.ASZ7UO38XJWD1IDZOPB6EOEC3NIUCW'
client_secret = 'b6dd7c3bc0d313249fe809151e05ce2a9f099ec762'
organization_id = '60026691546'
redirect_uri = 'http://localhost:8000/callback'
scope = 'ZohoBooks.fullaccess.all,ZohoInventory.fullaccess.all'
auth_url = (
    f"https://accounts.zoho.com/oauth/v2/auth?"
    f"response_type=code&client_id={client_id}&"
    f"scope={scope}&redirect_uri={redirect_uri}&"
    f"access_type=offline&prompt=consent"
)
token_url = "https://accounts.zoho.in/oauth/v2/token"

def start_server():
    server = socketserver.TCPServer(("", PORT), Handler)
    thread = threading.Thread(target=server.serve_forever)
    thread.daemon = True
    thread.start() 
    return server, thread

def stop_server(server, thread):
    server.shutdown()
    server.server_close()
    thread.join()

def authorize():
    chromedriver_autoinstaller.install()
    options = webdriver.ChromeOptions() 
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=options)

    driver.get(auth_url)

    wait = WebDriverWait(driver, 10)

    try:
        login_id_input = wait.until(EC.presence_of_element_located((By.NAME, "LOGIN_ID")))
        login_id_input.send_keys("ayush@autoversemobility.in")
        login_id_input.send_keys(Keys.RETURN)

        password_input = wait.until(EC.element_to_be_clickable((By.NAME, "PASSWORD")))
        password_input.send_keys("Ayush@12527")
        password_input.submit()

        authorize_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]")))
        authorize_button.click()

        while not authorization_code:
            time.sleep(1)
    finally:
        driver.quit()

def get_tokens():
    global authorization_code
    server, thread = start_server()
    authorize()
    stop_server(server, thread)

    data = {
        'grant_type': 'authorization_code',
        'client_id': client_id,
        'client_secret': client_secret,
        'redirect_uri': redirect_uri,
        'code': authorization_code
    }
    response = requests.post(token_url, data=data)

    if response.status_code == 200:
        tokens = response.json()
        if 'access_token' in tokens:
            return tokens['access_token'], tokens['refresh_token']
        else:
            print("Failed to retrieve access token. Response:", tokens)
            return None, None
    else:
        print(f"Failed to exchange authorization code for tokens: {response.status_code} - {response.text}")
        return None, None

def refresh_access_token(refresh_token):
    data = {
        'grant_type': 'refresh_token',
        'client_id': client_id,
        'client_secret': client_secret,
        'refresh_token': refresh_token
    }
    response = requests.post(token_url, data=data)
    if response.status_code == 200:
        tokens = response.json()
        if 'access_token' in tokens:
            return tokens['access_token']
        else:
            print("Failed to retrieve access token. Response:", tokens)
            return None
    else:
        print(f"Failed to refresh access token: {response.status_code} - {response.text}")
        return None

# Load the refresh token from a file if available
try:
    with open('refresh_token.json', 'r') as f:
        refresh_token = json.load(f).get('refresh_token')
except FileNotFoundError:
    refresh_token = None

if refresh_token:
    access_token = refresh_access_token(refresh_token)
    if not access_token:
        access_token, refresh_token = get_tokens()
else:
    access_token, refresh_token = get_tokens()

# Save the new refresh token to a file
if refresh_token:
    with open('refresh_token.json', 'w') as f:
        json.dump({'refresh_token': refresh_token}, f)

if not access_token:
    print("Failed to obtain access token. Exiting.")
    exit(1)

# Function to fetch data from Zoho and save it to an Excel file
def fetch_data(url, headers, file_name):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        content_type = response.headers['Content-Type']
        if 'application/json' in content_type:
            json_data = response.json()
            if 'inventory_valuation' in json_data:
                df = pd.json_normalize(json_data['inventory_valuation'])
            else:
                print("JSON data does not contain 'inventory_valuation'")
                return False
            df.to_excel(file_name, index=False)
        elif ('application/vnd.openxmlformats-officedocument.spreadsheetml.sheet' in content_type or 
              'text/xlsx' in content_type or
              'application/vnd.ms-excels;charset=UTF-8' in content_type):
            with open(file_name, 'wb') as file:
                file.write(response.content)
        else:
            print(f"Unexpected content type: {content_type}")
            return False
        print(f'Data has been saved to {file_name}')
        return True
    else:
        print(f'Failed to fetch data: {response.status_code} - {response.text}')
        return False

# Function to read Excel file and convert it to list of lists
def read_excel_to_list(file_path):
    df = pd.read_excel(file_path)
    return [df.columns.tolist()] + df.replace({np.nan: None}).values.tolist()

# Function to update Google Sheet with data
def update_google_sheet(sheet_id, range_name, data, credentials):
    sheets_service = build('sheets', 'v4', credentials=credentials)
    body = {'values': data}
    result = sheets_service.spreadsheets().values().update(
        spreadsheetId=sheet_id,
        range=range_name,
        valueInputOption='RAW',
        body=body
    ).execute()
    print(f'{result.get("updatedCells")} cells updated in the Google Sheet.')

# URLs and file names
url1 = [
    'https://books.zoho.in/api/v3/reports/invoicedetails?accept=xlsx&page=1&per_page=50000&sort_column=date&sort_order=A&filter_by=InvoiceDate.CustomDate&from_date=2024-02-01&to_date=2024-06-30&select_columns=%5B%7B%22field%22%3A%22status%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22date%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22due_date%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22invoice_number%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22reference_number%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22customer_name%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22bcy_total%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22bcy_balance%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22last_payment_date%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22payment_terms_label%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22warehouse_name%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22bcy_amount_paid%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22custom_field_1661984000003539033%22%2C%22group%22%3A%22contact%22%7D%2C%7B%22field%22%3A%22custom_field_1661984000010769297%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22created_by%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22credit_amount%22%2C%22group%22%3A%22invoice%22%7D%5D&group_by=%5B%7B%22field%22%3A%22none%22%2C%22group%22%3A%22report%22%7D%5D&usestate=true&response_option=1&x-zb-source=zbclient&formatneeded=true&is_header_and_footer_enabled=true&show_current_view=false&paper_size=A4&orientation=portrait&font_family_for_body=opensans&margin_top=0.7&margin_bottom=0.7&margin_left=0.55&margin_right=0.2&table_size=compact&table_style=alternative_rows&show_org_name=true&show_generated_date=false&show_generated_time=false&show_page_number=false&show_report_basis=true&show_generated_by=false&can_fit_to_page=true&organization_id=60026691546&frameorigin=https%3A%2F%2Fbooks.zoho.in'
]

url2 = [
    'https://books.zoho.in/api/v3/export?entity=salesorder&accept=xlsx&async=false&filter_by=OrderDate.CustomDate&from_date=2024-05-01&to_date=2024-06-30&Status.All&sort_column=created_time&sort_order=D&can_export_pii_fields=false&export_template_id=&decimal_format=.&x-zb-source=zbclient&organization_id=60026691546&frameorigin=https%3A%2F%2Fbooks.zoho.in'
]

url3 = [
    'https://books.zoho.in/api/v3/export?entity=creditnote&accept=xlsx&async=false&status=all&can_export_pii_fields=false&from_date=2024-02-01&to_date=2024-06-30&export_template_id=&decimal_format=.&x-zb-source=zbclient&organization_id=60026691546&frameorigin=https%3A%2F%2Fbooks.zoho.in'
]

url4 = [
    'https://books.zoho.in/api/v3/export?entity=invoice&accept=xlsx&async=false&status=all&can_export_pii_fields=false&from_date=2024-06-01&to_date=2024-06-30&export_template_id=&decimal_format=.&x-zb-source=zbclient&organization_id=60026691546&frameorigin=https%3A%2F%2Fbooks.zoho.in'
]

url5 = [
    'https://books.zoho.in/api/v3/reports/fifocostlottracking?accept=xls&page=1&per_page=50000&filter_by=TransactionDate.CustomDate&from_date=2024-06-01&to_date=2024-10-31&response_option=1&x-zb-source=zbclient&formatneeded=true&is_header_and_footer_enabled=true&paper_size=A4&orientation=portrait&font_family_for_body=opensans&margin_top=0.7&margin_bottom=0.7&margin_left=0.55&margin_right=0.2&table_size=compact&table_style=alternative_rows&show_org_name=true&show_generated_date=false&show_generated_time=false&show_page_number=false&show_report_basis=true&show_generated_by=false&can_fit_to_page=true&organization_id=60026691546&frameorigin=https%3A%2F%2Fbooks.zoho.in'
]

url6 = [
    'https://books.zoho.in/api/v3/export?entity=salesorder&accept=xlsx&async=false&filter_by=OrderDate.CustomDate&from_date=2024-06-01&to_date=2024-06-30&Status.All&sort_column=created_time&sort_order=D&can_export_pii_fields=false&export_template_id=&decimal_format=.&x-zb-source=zbclient&organization_id=60026691546&frameorigin=https%3A%2F%2Fbooks.zoho.in'
]

url7 = [
    'https://books.zoho.in/api/v3/reports/invoicedetails?accept=xlsx&page=1&per_page=20000&sort_column=date&sort_order=A&filter_by=InvoiceDate.ThisMonth&rule=%7B%22columns%22%3A%5B%7B%22index%22%3A1%2C%22field%22%3A%22status%22%2C%22value%22%3A%5B%22draft%22%2C%22overdue%22%2C%22paid%22%2C%22sent%22%2C%22partially_paid%22%5D%2C%22comparator%22%3A%22in%22%2C%22group%22%3A%22invoice%22%7D%5D%2C%22criteria_string%22%3A%221%22%7D&select_columns=%5B%7B%22field%22%3A%22status%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22date%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22due_date%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22invoice_number%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22reference_number%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22customer_name%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22bcy_total%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22bcy_balance%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22last_payment_date%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22payment_terms_label%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22warehouse_name%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22bcy_amount_paid%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22custom_field_1661984000003539033%22%2C%22group%22%3A%22contact%22%7D%2C%7B%22field%22%3A%22custom_field_1661984000010769297%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22created_by%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22credit_amount%22%2C%22group%22%3A%22invoice%22%7D%2C%7B%22field%22%3A%22custom_field_1661984000000022356%22%2C%22group%22%3A%22invoice%22%7D%5D&group_by=%5B%7B%22field%22%3A%22none%22%2C%22group%22%3A%22report%22%7D%5D&usestate=true&response_option=1&x-zb-source=zbclient&formatneeded=true&is_header_and_footer_enabled=true&show_current_view=false&paper_size=A4&orientation=portrait&font_family_for_body=opensans&margin_top=0.7&margin_bottom=0.7&margin_left=0.55&margin_right=0.2&table_size=compact&table_style=alternative_rows&show_org_name=true&show_generated_date=false&show_generated_time=false&show_page_number=false&show_report_basis=true&show_generated_by=false&can_fit_to_page=true&organization_id=60026691546&frameorigin=https%3A%2F%2Fbooks.zoho.in'
]

url8 = [
    'https://books.zoho.in/api/v3/reports/activitylogs?accept=xlsx&page=1&per_page=20000&sort_column=date&sort_order=D&filter_by=CreatedDate.ThisMonth&select_columns=%5B%7B%22field%22%3A%22date%22%2C%22group%22%3A%22report%22%7D%2C%7B%22field%22%3A%22transaction_type%22%2C%22group%22%3A%22report%22%7D%2C%7B%22field%22%3A%22description%22%2C%22group%22%3A%22report%22%7D%5D&rule=%7B%22columns%22%3A%5B%7B%22index%22%3A1%2C%22field%22%3A%22transaction_type%22%2C%22value%22%3A%5B%22invoice%22%5D%2C%22comparator%22%3A%22in%22%2C%22group%22%3A%22report%22%7D%5D%2C%22criteria_string%22%3A%221%22%7D&response_option=0&x-zb-source=zbclient&formatneeded=true&is_header_and_footer_enabled=true&paper_size=A4&orientation=portrait&font_family_for_body=opensans&margin_top=0.7&margin_bottom=0.7&margin_left=0.55&margin_right=0.2&table_size=compact&table_style=alternative_rows&show_org_name=true&show_generated_date=false&show_generated_time=false&show_page_number=false&show_report_basis=true&show_generated_by=false&can_fit_to_page=true&organization_id=60026691546&frameorigin=https%3A%2F%2Fbooks.zoho.in'
]

file_names = ['Excel Data/Invoice.xlsx', 'Excel Data/Sales_order.xlsx', 'Excel Data/Credit_Note.xlsx', 'Excel Data/Margin_Inv.xlsx', 'Excel Data/FIFO_ACPL.xlsx', 'Excel Data/SO_for_Delivery.xlsx', 'Excel Data/Inv_for_Delivery.xlsx', 'Excel Data/Activity_for_Delivery.xlsx']
sheet_ids = ['1plg6hb7eynJuwDtcrFkRiJ0yoTY9nr5pG2BjcI44MuU', '1jY15FiVrfu1iwT6SdK69ETThD-WVP_SAlYpnbjwXh90', '1plg6hb7eynJuwDtcrFkRiJ0yoTY9nr5pG2BjcI44MuU', '1UTLkxdLmH28NJX17wY170vOeTK1Suvhl-KvSE-__6BQ', '1UTLkxdLmH28NJX17wY170vOeTK1Suvhl-KvSE-__6BQ', '1eBj6JV9id8XjU6REuJSAesVO8u3UXLvUwBC0ssVuWnE', '1eBj6JV9id8XjU6REuJSAesVO8u3UXLvUwBC0ssVuWnE', '1eBj6JV9id8XjU6REuJSAesVO8u3UXLvUwBC0ssVuWnE']
sheet_ranges = [['Sheet1!A1'], ['SO_Acpl!A1'], ['Credit Note!A1'], ['INV_ACPL!A1'], ['FIFO_ACPL!A1'], ['SO_ACPL!A1'], ['Invoice_ACPL!A1'], ['Activity_Zoho!A1']]

# Google Sheets API setup
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'Zoho Report/Googlecred.json'  
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

# Process each URL and update respective sheet
for url_list, file_name, sheet_id, sheet_range_list in zip([url1, url2, url3, url4, url5, url6, url7, url8], file_names, sheet_ids, sheet_ranges):
    for url, sheet_range in zip(url_list, sheet_range_list):
        if fetch_data(url, {'Authorization': f'Zoho-oauthtoken {access_token}', 'Content-Type': 'application/json'}, file_name):
            data = read_excel_to_list(file_name)
            update_google_sheet(sheet_id, sheet_range, data, credentials)

Data has been saved to Excel Data/Invoice.xlsx


C:\Users\syeda\AppData\Roaming\Python\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


141624 cells updated in the Google Sheet.
Failed to fetch data: 429 - {"code":43,"message":"For security reasons you have been blocked for some time as you have exceeded the maximum number of requests per minute."}
Data has been saved to Excel Data/Credit_Note.xlsx


C:\Users\syeda\AppData\Roaming\Python\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


33544 cells updated in the Google Sheet.
Data has been saved to Excel Data/Margin_Inv.xlsx


...

444435 cells updated in the Google Sheet.
Data has been saved to Excel Data/FIFO_ACPL.xlsx
39707 cells updated in the Google Sheet.
Failed to fetch data: 429 - {"code":43,"message":"For security reasons you have been blocked for some time as you have exceeded the maximum number of requests per minute."}
Data has been saved to Excel Data/Inv_for_Delivery.xlsx


C:\Users\syeda\AppData\Roaming\Python\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


38380 cells updated in the Google Sheet.
Data has been saved to Excel Data/Activity_for_Delivery.xlsx


C:\Users\syeda\AppData\Roaming\Python\Python311\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


113034 cells updated in the Google Sheet.
